In [4]:
import os
# os.chdir('../py')
import gc
import sys
import warnings
warnings.filterwarnings('ignore')
from copy import deepcopy
from glob import glob

import feather
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams['font.family'] = 'YuGothic'
sns.set(style="whitegrid", font='YuGothic', palette="muted",
        color_codes=True, rc={'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

In [5]:
#前処理
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [6]:
#各種機械学習モデル
#LogisticRegression
from sklearn.linear_model import LogisticRegression
#Linear Support Vector Machine
from sklearn.svm import LinearSVC
#Support Vector Machine
from sklearn.svm import SVC
#決定木
from sklearn.tree import DecisionTreeClassifier
#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
#k近傍方（k-neighbor）
from sklearn.neighbors import KNeighborsClassifier

#学習方法（GridSearchCV）
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

#scikiti-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,GroupKFold
from sklearn.metrics import roc_auc_score

#スコアリング（f値）
from sklearn.metrics import f1_score

#lightgbm
import lightgbm as lgb
import pandas_profiling as pdp

/Users/yuki.matsumoto/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [48]:
#TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [10]:
docs = np.array([
        '牛乳        を 買う',
        'パン         を 買う',
        'パン         を 食べる',
        'お菓子       を 食べる',
        '本           を 買う',
        'パン と お菓子 を 食べる',
        'お菓子        を 買う',
        'パン と パン   を 食べる'
        ])

In [13]:
# TfidVextrizer
#ベクトル化
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b')
vecs = vectorizer.fit_transform(docs)
print(vecs.toarray())

[[0.         0.         0.31957739 0.         0.         0.80024653
  0.50742072 0.        ]
 [0.         0.         0.4068224  0.64594719 0.         0.
  0.64594719 0.        ]
 [0.         0.         0.4068224  0.64594719 0.         0.
  0.         0.64594719]
 [0.69443273 0.         0.38346742 0.         0.         0.
  0.         0.60886445]
 [0.         0.         0.31957739 0.         0.80024653 0.
  0.50742072 0.        ]
 [0.48880235 0.56645286 0.26991783 0.42857192 0.         0.
  0.         0.42857192]
 [0.69443273 0.         0.38346742 0.         0.         0.
  0.60886445 0.        ]
 [0.         0.4945171  0.23564005 0.74829225 0.         0.
  0.         0.37414612]]


In [16]:
# クラスタリング
clusters = KMeans(n_clusters=2, random_state=0).fit_predict(vecs)
for doc, cls in zip(docs, clusters):
    print(cls, doc)

0 牛乳        を 買う
0 パン         を 買う
1 パン         を 食べる
1 お菓子       を 食べる
0 本           を 買う
1 パン と お菓子 を 食べる
0 お菓子        を 買う
1 パン と パン   を 食べる


### 文章の前処理

テキストのベクトル化

In [50]:
#CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
sample = np.array(['Apple computer of the apple mark', 'linux computer', 'windows computer'])

In [53]:
#CountVectorizer
vec_count = CountVectorizer()
#ベクトル化
vec_count.fit(sample)
trans_vec = vec_count.transform(sample)
print('Vocabulary size: {}'.format(len(vec_count.vocabulary_)))
print('Vocabulary content: {}'.format(vec_count.vocabulary_))

Vocabulary size: 7
Vocabulary content: {'apple': 0, 'computer': 1, 'of': 4, 'the': 5, 'mark': 3, 'linux': 2, 'windows': 6}


In [55]:
pd.DataFrame(trans_vec.toarray(), columns=vec_count.get_feature_names())

,apple,computer,linux,mark,of,the,windows
0,2,1,0,1,1,1,0
1,0,1,1,0,0,0,0
2,0,1,0,0,0,0,1


これだと出現頻度が高い単語は強くなる
次はTF-IDF

In [61]:
#TfidfVectorizer
vec_tfidf = TfidfVectorizer()
#ベクトル化
trans_vec_tfidf = vec_tfidf.fit_transform(sample)

print('Vocabulary size: {}'.format(len(vec_tfidf.vocabulary_)))
print('Vocabulary content: {}'.format(vec_tfidf.vocabulary_))

Vocabulary size: 7
Vocabulary content: {'apple': 0, 'computer': 1, 'of': 4, 'the': 5, 'mark': 3, 'linux': 2, 'windows': 6}


In [62]:
pd.DataFrame(trans_vec_tfidf.toarray(), columns=vec_tfidf.get_feature_names())

,apple,computer,linux,mark,of,the,windows
0,0.73777,0.217869,0.000000,0.368885,0.368885,0.368885,0.000000
1,0.00000,0.508542,0.861037,0.000000,0.000000,0.000000,0.000000
2,0.00000,0.508542,0.000000,0.000000,0.000000,0.000000,0.861037


Word2Vecしてみる

In [63]:
import MeCab

class Wakati(object):

    """
    ==========================================================
    ファイルに存在する文章を指定の辞書を用いてMeCabによって形態素に分ける
    ==========================================================
    【関数説明】
    __init__ : コンストラクタ
    wakati : 文章を分かち書きする
    output : 結果をファイルに出力する
    """

    def __init__(self, file_dir, dic_dir=None, user_dir=None, hinshis=["動詞", "形容詞", "形容動詞", "助動詞"]):
        """
        ==========================================================
        コンストラクタ
        ==========================================================
        【変数説明】
        file_dir : 入力となる文章のディレクトリ
        dic_dir : システム辞書のディレクトリ([ex] /usr/local/lib/mecab/dic/mecab-ipadic-neologd)
        user_dir : ユーザー辞書のディレクトリ([ex] /Users/PCのユーザ名/...)
        hinshis : 活用する語
        tagger : MeCab用のtagger(詳細はMeCab自体のドキュメント参照)
        f : 入力ファイル
        splited_text : 各行を分かち書きしたもの(splited_line)をリストで格納する(Noneで初期化)
        out_dir : 出力ファイルのディレクトリ(Noneで初期化)
        """
        if dic_dir is not None and user_dir is not None:
            self.tagger = MeCab.Tagger("mecabrc -d {} -u {}".format(dic_dir, user_dir))
        elif dic_dir is not None:
            self.tagger = MeCab.Tagger("mecabrc -d {}".format(dic_dir))
        else:
            self.tagger = MeCab.Tagger("mecabrc")
        self.f = open(file_dir, 'r')
        self.hinshis = hinshis
        self.splited_text = None
        self.out_dir = None

    def wakati(self):
        """
        ==========================================================
        文章全体を分かち書きし、self.splited_textに格納する
        その際、活用された語については原形に直す
        ==========================================================
        【変数説明】
        line : 入力文章の一行(更新されていく)
        splited_line : 各行の文章を分かち書きしたもののリスト
        node : 各単語のノード
        word : 単語
        feature : 単語の情報
        hinshi : 品詞
        kata : 活用形
        genkei : 原形
        """
        line = self.f.readline()
        splited_text = []
        while line:
            node = self.tagger.parseToNode(line).next
            splited_line = []
            while node.surface:
                word = node.surface
                feature = node.feature.split(',')
                hinshi = feature[0]
                kata = feature[5]
                genkei = feature[6]
                if hinshi in self.hinshis:
                    if kata != "基本形":
                        word = genkei
                splited_line.append(word)
                node = node.next
            splited_text.append(splited_line)
            line = self.f.readline()
        self.splited_text = splited_text
        self.f.close()

    def output(self, out_dir):
        """
        ==========================================================
        self.splited_textをファイルに出力する
        ==========================================================
        【変数説明】
        out_dir : 出力ファイルのディレクトリ
        fout : 出力ファイル
        """
        assert self.splited_text is not None
        if self.out_dir is None:
            self.out_dir = out_dir
        self.fout = open(self.out_dir, 'w')
        for line in self.splited_text:
            self.fout.write(" ".join(line) + " ")
        self.fout.close()

In [ ]:
from gensim.models import word2vec
import logging

class Vectorizer(Wakati):

    """
    ==========================================================
    Wakatiをベースとして、分かち書きしたものを学習して分散表現する
    ==========================================================
    【関数説明】
    __init__ : コンストラクタ
    vectorize : 分散表現を作る
    _train : gensimを使ってword2vecする
    save_model : 作ったモデルを保存する
    """

    def __init__(self, file_dir, dic_dir=None, user_dir=None, out_dir="out.txt", hinshis=["動詞", "形容詞", "形容動詞", "助動詞"]):
        """
        ==========================================================
        コンストラクタ
        Wakatiを使って文章を分かち書きしておく
        ==========================================================
        【変数説明】
        file_dir : 入力となる文章のディレクトリ
        dic_dir : システム辞書のディレクトリ(/usr/local/lib/mecab/dic/mecab-ipadic-neologd)
        user_dir : ユーザー辞書のディレクトリ(/Users/ユーザ名/Desktop/word2vec/user.dic)
        out_dir : 分かち書きされた文章のファイルのディレクトリ
        hinshis : 活用する語
        model : モデル(Noneで初期化)
        """
        Wakati.__init__(self, file_dir, dic_dir, user_dir, hinshis)
        self.out_dir = out_dir
        self.model = None
        self.wakati()
        self.output(self.out_dir)

    def vectorize(self, other_file_dir=None, sg=1, size=300, min_count=10, window=5, hs=0, negative=15, iter=15):
        """
        ==========================================================
        単語の分散表現を作成
        ==========================================================
        【変数説明】
        out_dir : 分かち書きされた文章のファイル
        other_file_dir : out_dirを使わない場合のファイル名(Noneで初期化)
        sentences : 分かち書きされ、空白区切の文章全文
        """
        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
        if other_file_dir is None:
            sentences = word2vec.Text8Corpus(self.out_dir)
        else:
            sentences = word2vec.Text8Corpus(other_file_dir)
        self._train(sentences, sg, size, min_count, window, hs, negative, iter)

    def _train(self, sentences, sg=1, size=300, min_count=10, window=5, hs=0, negative=15, iter=15):
        """
        ==========================================================
        gensimによる学習
        ==========================================================
        【変数説明】
        sentences : 分かち書きされ、空白区切の文章全文
        word2vecの引数 : 詳細はgensimのドキュメント参照
        """
        self.model = word2vec.Word2Vec(sentences, sg=sg, size=size, min_count=min_count, window=window, hs=hs, negative=negative, iter=iter)

    def save_model(self, model_dir):
        """
        ==========================================================
        モデルを保存する
        ==========================================================
        【変数説明】
        model_dir : モデルを保存するディレクトリ
        """
        assert self.model is not None
        self.model.save(model_dir)

In [ ]:
from Wakati import Wakati

w = Wakati("kokoro.txt", システム辞書のディレクトリ, ユーザ辞書のディレクトリ)
w.wakati()
w.output("out.txt")

In [ ]:
from Vectorizer import Vectorizer

"""
==========================================================
自作のWord2Vecを使って、夏目漱石の『こころ』を解析する。
(『こころ』は青空文庫(http://www.aozora.gr.jp/cards/000148/card773.html)から
ダウンロードして同じディレクトリに入れておく)
==========================================================
【関数説明】
kokoro : 『こころ』のモデルを作成
"""

def kokoro():
    """
    ==========================================================
    『こころ』から作成されたモデルを返す
    ==========================================================
    【変数説明】
    dic_dir : システム辞書のディレクトリ(このPCでは"/usr/local/lib/mecab/dic/mecab-ipadic-neologd")
    user_dir : ユーザー辞書のディレクトリ(このPCでは"/Users/ユーザ名/Desktop/word2vec/user.dic")
    """

    #Vectorizerインスタンスを作成
    #__init__で文章が分かち書きされたファイル(out.txt)が同じディレクトリ内に作られる。
    #dic_dir、user_dirを各自のPCに合わせて設定し、Vectorizerの引数にもたせても良い。
    v = Vectorizer(file_dir="kokoro.txt")

    #単語の分散表現のモデル作成
    v.vectorize()

    #できたモデルを返す
    return v.model

if __name__ == "__main__":

    #『こころ』から作成されたモデル
    model = kokoro()

    #「人間」という単語に最も近い１０単語を表示する
    #resultは(単語, コサイン距離)からなるリスト
    result = model.most_similar(positive="人間")
    for pair in result:
        word = pair[0]
        distance = pair[1]
        print(word, distance)

    #完成したモデルを保存したければ以下のコードのコメントアウトを外す
    #v.save_model("kokoro.model")


「学問のススメ」を解析してみる

In [27]:
#Mecabで分割
from pathlib import Path
import MeCab

data_dir_path = Path('.').joinpath('input')

In [41]:
tagger = MeCab.Tagger("-Ochasen")
tagger.parse('')

#  「学問のすすめ」のデータを読み込む
with open(data_dir_path.joinpath('gakumonno_susume.txt'), 'r', encoding='shift-jis') as file:
    lines = file.readlines()

sentences = []
for sentence in lines:
    texts = sentence.split('。')
    sentences.extend(texts)
    
sentence = sentences[23]
words = tagger.parse(sentence)

print('元の文： ', sentence)
print('')
print('MeCabにより分割後')
print(words)

元の文：  「天は人の上に人を造らず人の下に人を造らず」と言えり

MeCabにより分割後
「	「	「	記号-括弧開		
天	テン	天	名詞-一般		
は	ハ	は	助詞-係助詞		
人	ヒト	人	名詞-一般		
の	ノ	の	助詞-連体化		
上	ウエ	上	名詞-非自立-副詞可能		
に	ニ	に	助詞-格助詞-一般		
人	ヒト	人	名詞-一般		
を	ヲ	を	助詞-格助詞-一般		
造ら	ツクラ	造る	動詞-自立	五段・ラ行	未然形
ず	ズ	ぬ	助動詞	特殊・ヌ	連用ニ接続
人	ヒト	人	名詞-一般		
の	ノ	の	助詞-連体化		
下	シタ	下	名詞-一般		
に	ニ	に	助詞-格助詞-一般		
人	ヒト	人	名詞-一般		
を	ヲ	を	助詞-格助詞-一般		
造ら	ツクラ	造る	動詞-自立	五段・ラ行	未然形
ず	ズ	ぬ	助動詞	特殊・ヌ	連用ニ接続
」	」	」	記号-括弧閉		
と	ト	と	助詞-格助詞-引用		
言え	イエ	言える	動詞-自立	一段	連用形
り	リ	り	助動詞	文語・リ	基本形
EOS



文においてはそれぞれの数字の値に意味がある場合もありますが、単語の関連性を分析するうえでは数字は数字として一つにまとめた方が良い

### Word Cloud

文章の単語出現頻度を可視化